In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import sem
import seaborn as sns
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics
import warnings as warnings
from sklearn import preprocessing
from matplotlib.offsetbox import AnchoredText

In [ ]:
%pwd

In [ ]:
BL_all=pd.read_csv("../PPMI_Dataset/Evolution/PD_BaseLine_Features.csv")
BL_all.drop(columns="Unnamed: 0",axis=1,inplace=True)
V04=pd.read_csv("../PPMI_Dataset/Evolution/V04_Features.csv")
V04.drop(columns="Unnamed: 0",axis=1,inplace=True)
V06=pd.read_csv("../PPMI_Dataset/Evolution/V06_Features.csv")
V06.drop(columns="Unnamed: 0",axis=1,inplace=True)
V08=pd.read_csv("../PPMI_Dataset/Evolution/V08_Features.csv")
V08.drop(columns="Unnamed: 0",axis=1,inplace=True)
V10=pd.read_csv("../PPMI_Dataset/Evolution/V10_Features.csv")
V10.drop(columns="Unnamed: 0",axis=1,inplace=True)
V12=pd.read_csv("../PPMI_Dataset/Evolution/V12_Features.csv")
V12.drop(columns="Unnamed: 0",axis=1,inplace=True)


In [ ]:
def scores(x,period):
    mds_updrs4=pd.read_csv('../PPMI_Dataset/Motor_Assessments/MDS_UPDRS_Part_IV.csv')
    
    
    df=x.iloc[:,np.r_[0,43:102,183]].copy()
    
    
    mds_updrs4=mds_updrs4[mds_updrs4['EVENT_ID']==period]
    
    
    df['Cognitive']=df.loc[:,['NP1COG','NP1HALL','NP1DPRS','NP1ANXS','NP1APAT','NP1DDS']].sum(axis=1,skipna=False)
    df['Sleep']=df.loc[:,['NP1SLPN','NP1SLPD']].sum(axis=1,skipna=False)
    df['Autonomic_Nervous_System']=df.loc[:,['NP1URIN','NP1CNST','NP1LTHD','NP1FATG']].sum(axis=1,skipna=False)
    df['Total_UPDRS1']=df.loc[:,'NP1COG':'NP1FATG'].sum(axis=1,skipna=False)
    
    df['Bulbar']=df.loc[:,['NP2SPCH','NP2SALV','NP2SWAL','NP2EAT']].sum(axis=1,skipna=False)
    df['Common_daily_act']= df.loc[:,['NP2DRES','NP2HYGN','NP2HWRT','NP2HOBB']].sum(axis=1,skipna=False)
    df['Bed']=df.loc[:,['NP2TURN','NP2RISE']].sum(axis=1,skipna=False)
    df['Gait']=df.loc[:,['NP2WALK','NP2FREZ']].sum(axis=1,skipna=False)
    df['Total_UPDRS2']=df.loc[:,'NP2SPCH':'NP2FREZ'].sum(axis=1,skipna=False)
    
    
    df['Axial_Sub_1']=df.loc[:,['NP3SPCH','NP3FACXP','NP3RIGN',
                         'NP3RISNG','NP3GAIT','NP3FRZGT',
                         'NP3PSTBL','NP3POSTR','NP3BRADY','NP3RTALJ']].sum(axis=1,skipna=False)
    df['Axial_Sub_2']=df.loc[:,['NP3RISNG','NP3GAIT','NP3FRZGT',
                         'NP3PSTBL','NP3POSTR','NP3BRADY']].sum(axis=1,skipna=False)
    df['Limb_Rig_Sub']=df.loc[:,['NP3RIGRU','NP3RIGLU','PN3RIGRL','NP3RIGLL']].sum(axis=1,skipna=False)
    df['Limb_Brady_Sub']=df.loc[:,['NP3FTAPR','NP3FTAPL','NP3HMOVR','NP3HMOVL',
                                 'NP3PRSPR','NP3PRSPL','NP3TTAPR','NP3TTAPL',
                                'NP3LGAGR','NP3LGAGL']].sum(axis=1,skipna=False)
    df['Tremor_Sub']=df.loc[:,['NP3PTRMR','NP3PTRML','NP3KTRMR','NP3KTRML','NP3RTARU',
                              'NP3RTALU','NP3RTARL','NP3RTALL','NP3RTCON']].sum(axis=1,skipna=False)
    df['Rest_Tremor_Sub']=df.loc[:,['NP3RTARU','NP3RTALU','NP3RTARL','NP3RTALL','NP3RTCON']].sum(axis=1,skipna=False)
    df['Append_Sub']=df.loc[:,['Limb_Rig_Sub','Limb_Brady_Sub','Tremor_Sub']].sum(axis=1,skipna=False)
    df['Left_Motor_Score']=df.loc[:,['NP3RIGLU','NP3RIGLL','NP3FTAPL','NP3HMOVL',
                                     'NP3PRSPL','NP3TTAPL','NP3LGAGL','NP3PTRML',
                                     'NP3KTRML','NP3RTALU','NP3RTALL']].sum(axis=1,skipna=False)
    #df['Left_Motor_Score']=df.loc[:,['NP3RIGLU','NP3RIGLL','NP3FTAPL','NP3HMOVL',
    #                                'NP3PRSPL','NP3TTAPL','NP3LGAGL']].sum(axis=1,skipna=False)
    
    df['Right_Motor_Score']=df.loc[:,['NP3RIGRU','PN3RIGRL','NP3FTAPR','NP3HMOVR',
                                      'NP3PRSPR','NP3TTAPR','NP3LGAGR','NP3PTRMR',
                                      'NP3KTRMR','NP3RTARU','NP3RTARL']].sum(axis=1,skipna=False)
    
    #df['Right_Motor_Score']=df.loc[:,['NP3RIGRU','PN3RIGRL','NP3FTAPR','NP3HMOVR',
    #                                 'NP3PRSPR','NP3TTAPR','NP3LGAGR']].sum(axis=1,skipna=False)
    
    df['Diff']=df['Left_Motor_Score']-df['Right_Motor_Score']
    df['Asymmetry']=np.abs(df['Diff'])
    df['Total_UPDRS3']=df.loc[:,'NP3SPCH':'NP3RTCON'].sum(axis=1,skipna=False)
    #df['Total_UPDRS3']=df.loc[:,'NP3SPCH':'NP3BRADY'].sum(axis=1,skipna=False)
    
    df['Total_UPDRS']=df['Total_UPDRS1']+df['Total_UPDRS2']+df['Total_UPDRS3']
    
    
    if mds_updrs4.empty==True:
        df['Total_UPDRS4']=np.nan
    else:
        mds_updrs4['Total_UPDRS4']=mds_updrs4.loc[:,'NP4WDYSK':'NP4DYSTN'].sum(axis=1,skipna=False)
        df=df.merge(mds_updrs4.loc[:,['PATNO','Total_UPDRS4']],on='PATNO',how='left')
        
    if period=='BL':
        
        temp=(df.loc[:,['NP2TRMR','NP3PTRMR',
                                       'NP3PTRML','NP3KTRMR',
                                       'NP3KTRML','NP3RTARU',
                                       'NP3RTALU','NP3RTARL',
                                       'NP3RTALL','NP3RTALJ',
                                       'NP3RTCON']].mean(axis=1,skipna=False))/(df.loc[:,['NP2WALK','NP2FREZ','NP3GAIT',
                                       'NP3FRZGT','NP3PSTBL']].mean(axis=1,skipna=False))
        pigd=[]
        for i in temp:
            if i<=1:
                pigd.append('PIGD')
            elif i >=1.5:
                pigd.append('TD')
            elif i > 1 and i < 1.5:
                pigd.append('IND')
            else:
                pigd.append('NOTA')
        df['PIGD_TD']=pigd
            
    return df

    

    
    
    ############################# LEFT-RIGHT
"""    
    df['Limb_Rig_Left-Right']=(df['NP3RIGLU']+df['NP3RIGLL'])-(df['NP3RIGRU']+df['PN3RIGRL'])
    
    df['Upper_Limb_Rig']=df['NP3RIGRU']+df['NP3RIGLU']
    
    df['Lower_Limb_Rig']=df['PN3RIGRL']+df['NP3RIGLL']
    
    df['Upper_Limb_Rig_Left-Right']=df['NP3RIGLU']-df['NP3RIGRU']
    
    df['Lower_Limb_Rig_Left-Right']=df['NP3RIGLL']-df['PN3RIGRL']
    
    df['Fing_Tap_Total']=df.loc[:,['NP3FTAPR','NP3FTAPL']].sum(axis=1,skipna=False)
    
    df['Fing_Tap_Left-Right']=df['NP3FTAPL']-df['NP3FTAPR']
    
    df['Hand_Mov_Total']=df.loc[:,['NP3HMOVR','NP3HMOVL']].sum(axis=1,skipna=False)
    
    df['Hand_Mov_Left-Right']=df['NP3HMOVL']-df['NP3HMOVR']
    
    df['Pron_Sup_Total']=df.loc[:,['NP3PRSPR','NP3PRSPL']].sum(axis=1,skipna=False)
    
    df['Pron_Sup_Left-Right']=df['NP3PRSPL']-df['NP3PRSPR']
    
    df['Toe_Tapping_Total']=df.loc[:,['NP3TTAPR','NP3TTAPL']].sum(axis=1,skipna=False)
    
    df['Toe_Tapping_Left-Right']=df['NP3TTAPL']-df['NP3TTAPR']
    
    df['Leg_Agility_Total']=df.loc[:,['NP3LGAGR','NP3LGAGL']].sum(axis=1,skipna=False)
    
    df['Leg_Agility_Left-Right']=df['NP3LGAGL']-df['NP3LGAGR']
    
    df['Postural_Tremor_Total']=df.loc[:,['NP3PTRMR','NP3PTRML']].sum(axis=1,skipna=False)
    
    df['Postural_Tremor_Left-Right']=df['NP3PTRML']-df['NP3PTRMR']
    
    df['Kinetic_Tremor_Hand_Total']=df.loc[:,['NP3KTRMR','NP3KTRML']].sum(axis=1,skipna=False)
    
    df['Kinetic_Tremor_Hand_Left-Right']=df['NP3KTRML']-df['NP3KTRMR']
    
    df['Rest_Tremor_Upper_Total']=df.loc[:,['NP3RTARU','NP3RTALU']].sum(axis=1,skipna=False)
    
    df['Rest_Tremor_Upper_Left-Right']=df['NP3RTALU']-df['NP3RTARU']
    
    df['Rest_Tremor_Lower_Total']=df.loc[:,['NP3RTARL','NP3RTALL']].sum(axis=1,skipna=False)
    
    df['Rest_Tremor_Lower_Left-Right']=df['NP3RTALL']-df['NP3RTARL']
    
    df['Rest_Tremor_Left-Right']=(df['NP3RTALU']+df['NP3RTALL'])-(df['NP3RTARU']+df['NP3RTARL'])
    
"""    


In [ ]:
BL=scores(BL_all,'BL')
year1=scores(V04,'V04')
year2=scores(V06,'V06')
year3=scores(V08,'V08')
year4=scores(V10,'V10')
year5=scores(V12,'V12')

In [ ]:
file=V12
filenew=year5

print('Age_Man',np.mean(file[file['gen']==1]['age']))
print('STD_MAN',np.std(file[file['gen']==1]['age']))
print('AGE_Woman',np.mean(file[file['gen']==2]['age']))
print('STD_Woman',np.std(file[file['gen']==2]['age']))
print('1',np.mean(filenew['Total_UPDRS1']))
print('1_STD',np.std(filenew['Total_UPDRS1']))
print('2',np.mean(filenew['Total_UPDRS2']))
print('2_STD',np.std(filenew['Total_UPDRS2']))
print('3',np.mean(filenew['Total_UPDRS3']))
print('3_STD',np.std(filenew['Total_UPDRS3']))

In [ ]:
plt.hist(BL['Axial_Sub_2'])
#len(BL[BL['Asymmetry']>5])/423
plt.xlabel('Axial Index')
plt.ylabel('Patients number')
plt.savefig('Axial_Dist')

In [ ]:
from scipy.stats import mannwhitneyu
x=[BL,year1,year3,year5]
names=['BL','year1','year3','year5']
#y=BL_all[BL_all['PATNO'].isin(x['PATNO'])]
#pat=y[y['gen']==2]
#pat
for i in range(len(x)-1):
    print(names[i],names[i+1])
    print('Test on PartI:',mannwhitneyu(x[i]['Total_UPDRS1'].dropna(),x[i+1]['Total_UPDRS1'].dropna()))
    print('Test on PartII:',mannwhitneyu(x[i]['Total_UPDRS2'].dropna(),x[i+1]['Total_UPDRS2'].dropna()))
    print('Test on PartIII:',mannwhitneyu(x[i]['Total_UPDRS3'].dropna(),x[i+1]['Total_UPDRS3'].dropna()))

In [ ]:
Left_Aff=BL[BL['Diff']>0]
Right_Aff=BL[BL['Diff']<0]

print("Left affected",len(Left_Aff))
print("Right affected",len(Right_Aff))

In [ ]:
####################################### MOST AFFECTED-LESS AFFECTED
def affected(x):
    df=x.copy()
    
    R=df[df['PATNO'].isin(Right_Aff['PATNO'])].copy()
    L=df[df['PATNO'].isin(Left_Aff['PATNO'])].copy()
    
    
    L['Limb_Rig_Most-Less']=(L['NP3RIGLU']+L['NP3RIGLL'])-(L['NP3RIGRU']+L['PN3RIGRL'])
    R['Limb_Rig_Most-Less']=(R['NP3RIGRU']+R['PN3RIGRL'])-(R['NP3RIGLU']+R['NP3RIGLL'])
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Limb_Rig_Most-Less']=s['Limb_Rig_Most-Less']

    L['Upper_Limb_Rig_Most-Less']=L['NP3RIGLU']-L['NP3RIGRU']
    R['Upper_Limb_Rig_Most-Less']=R['NP3RIGRU']-R['NP3RIGLU']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Upper_Limb_Rig_Most-Less']=s['Upper_Limb_Rig_Most-Less']
    
    
    L['Lower_Limb_Rig_Most-Less']=L['NP3RIGLL']-L['PN3RIGRL']
    R['Lower_Limb_Rig_Most-Less']=R['PN3RIGRL']-R['NP3RIGLL']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Lower_Limb_Rig_Most-Less']=s['Lower_Limb_Rig_Most-Less']
    
    L['Fing_Tap_Most-Less']=L['NP3FTAPL']-L['NP3FTAPR']
    R['Fing_Tap_Most-Less']=R['NP3FTAPR']-R['NP3FTAPL']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Fing_Tap_Most-Less']=s['Fing_Tap_Most-Less']
    
    
    L['Hand_Mov_Most-Less']=L['NP3HMOVL']-L['NP3HMOVR']
    R['Hand_Mov_Most-Less']=R['NP3HMOVR']-R['NP3HMOVL']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Hand_Mov_Most-Less']=s['Hand_Mov_Most-Less']

    L['Pron_Sup_Most-Less']=L['NP3PRSPL']-L['NP3PRSPR']
    R['Pron_Sup_Most-Less']=R['NP3PRSPR']-R['NP3PRSPL']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Pron_Sup_Most-Less']=s['Pron_Sup_Most-Less']
    
    L['Toe_Tapping_Most-Less']=L['NP3TTAPL']-L['NP3TTAPR']
    R['Toe_Tapping_Most-Less']=R['NP3TTAPR']-R['NP3TTAPL']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Toe_Tapping_Most-Less']=s['Toe_Tapping_Most-Less']
    
    L['Leg_Agility_Most-Less']=L['NP3LGAGL']-L['NP3LGAGR']
    R['Leg_Agility_Most-Less']=R['NP3LGAGR']-R['NP3LGAGL']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Leg_Agility_Most-Less']=s['Leg_Agility_Most-Less']
    
    L['Postural_Tremor_Most-Less']=L['NP3PTRML']-L['NP3PTRMR']
    R['Postural_Tremor_Most-Less']=R['NP3PTRMR']-R['NP3PTRML']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Postural_Tremor_Most-Less']=s['Postural_Tremor_Most-Less']
    
    L['Kinetic_Tremor_Hand_Most-Less']=L['NP3KTRML']-L['NP3KTRMR']
    R['Kinetic_Tremor_Hand_Most-Less']=R['NP3KTRMR']-R['NP3KTRML']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Kinetic_Tremor_Hand_Most-Less']=s['Kinetic_Tremor_Hand_Most-Less']
    
    L['Rest_Tremor_Upper_Most-Less']=L['NP3RTALU']-L['NP3RTARU']
    R['Rest_Tremor_Upper_Most-Less']=R['NP3RTARU']-R['NP3RTALU']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Rest_Tremor_Upper_Most-Less']=s['Rest_Tremor_Upper_Most-Less']
    
    L['Rest_Tremor_Lower_Most-Less']=L['NP3RTALL']-L['NP3RTARL']
    R['Rest_Tremor_Lower_Most-Less']=R['NP3RTARL']-R['NP3RTALL']
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Rest_Tremor_Lower_Most-Less']=s['Rest_Tremor_Lower_Most-Less']
    
    L['Rest_Tremor_Most-Less']=(L['NP3RTALU']+L['NP3RTALL'])-(L['NP3RTARU']+L['NP3RTARL'])
    R['Rest_Tremor_Most-Less']=(R['NP3RTARU']+R['NP3RTARL'])-(R['NP3RTALU']+R['NP3RTALL'])
    s=pd.concat([L,R])
    s=s.sort_index(axis=0)
    df['Rest_Tremor_Most-Less']=s['Rest_Tremor_Most-Less']
    
    return df
    

In [ ]:
BL=affected(BL)
year1=affected(year1)
year2=affected(year2)
year3=affected(year3)
year4=affected(year4)
year5=affected(year5)

In [ ]:
x=

# Rule 1
### Diff≤2 : SYMMETRIC
### 3 < Diff <7 and  axial>3: medium ASYMM with not so low axial
### 3 < Diff <7 and   axial≤3: medium ASYMM with low axial
### Diff≥7: Highly Asymm

In [ ]:

Symm_Pat=BL[BL['Asymmetry']<=3]['PATNO']
print("Symmetric:",len(Symm_Pat))

Medium_asym_nolow_ax=BL[(BL['Asymmetry']>3) & (BL['Asymmetry']<7)&
                           (BL['Axial_Sub_2']>3)]['PATNO']

print("Medium asym with not so low axial:",len(Medium_asym_nolow_ax))


Medium_Asymm_low_axial=BL[(BL['Asymmetry']>3) & (BL['Asymmetry']<7)
                                            & (BL['Axial_Sub_2']<=3)]['PATNO']
print("Medium asym with low axial:",len(Medium_Asymm_low_axial))


High_Asymm=BL[BL['Asymmetry']>= 7]['PATNO']
print("Highly Asymmetric:",len(High_Asymm))


In [ ]:
p={'Group 1':57,'Group 2':30,'Group 3':86,'Group 4':250}
plt.figure(figsize=(5,5))
plt.bar(p.keys(),p.values())
plt.xticks(rotation=45, ha="right")
plt.savefig('Axial')


In [ ]:
Symm_Pat=BL[BL['Asymmetry']<=5]['PATNO']
print("Symmetric:",len(Symm_Pat))

ASymm_Pat=BL[BL['Asymmetry']>5]['PATNO']
print("Symmetric:",len(ASymm_Pat))


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
#pat_list=[Symm_Pat,Medium_asym_nolow_ax,Medium_Asymm_low_axial,High_Asymm]
pat_list=[Symm_Pat,ASymm_Pat]
years=[BL,year1,year2,year3,year4,year5]
for i in BL.iloc[:,np.r_[1,2,3,32,34:83]].columns:
    plt.figure(figsize=(10,5))
    plt.title(i)
    for j in pat_list:
        avg=[]
        serr=[]
        for k in years:
            patients=k[k['PATNO'].isin(j)][i]
            patients=patients[~np.isnan(patients)].to_list()
            avg.append(np.nanmean(patients))
            serr.append(sem(patients))
        a=[0]
        avg=a+[i-avg[0] for i in avg[1:]]  
        
        plt.plot(['BL','1','2','3','4','5'],avg,'-o')
        #plt.legend(['Symm','Medium_asym_nolow_ax','Medium_Asymm_low_axial','Highly Asymm'],loc='best')
        plt.legend(['Symm','Asymm'],loc='best')
        plt.fill_between(['BL','1','2','3','4','5'],np.array(avg)-np.array(serr),np.array(avg)+np.array(serr),alpha=0.2)
        plt.ylabel('Difference of AVG from BL')
        plt.xlabel('Years')
    if i =='Total_UPDRS3':
        plt.title('Motor Progression')
        plt.savefig(i)
    if i=='Total_UPDRS2':
        plt.title('Progression of Disability')
        plt.savefig(i)
    if i =='Total_UPDRS1':
        plt.title('Non-Motor Progression')
        plt.savefig(i)
        


In [ ]:
100*(6.06-5.93)/((6.06+5.93)/2)

In [ ]:
        
fig,axs=plt.subplots(1,2,figsize=(30,7))     
for j in pat_list:
    avg=[]
    serr=[]
    avg2=[]
    serr2=[]
    
    for k in years:
        
        patients=k[k['PATNO'].isin(j)]['Axial_Sub_2']
        patients=patients[~np.isnan(patients)].to_list()
        avg.append(np.nanmean(patients))
        serr.append(sem(patients))

        patients2=k[k['PATNO'].isin(j)]['Append_Sub']
        patients2=patients2[~np.isnan(patients2)].to_list()
        avg2.append(np.nanmean(patients2))
        serr2.append(sem(patients2))
            
    a=[0]
    avg=a+[i-avg[0] for i in avg[1:]] 
    
    avg2=a+[i-avg2[0] for i in avg2[1:]] 
    
    axs[0].plot(['BL','1','2','3','4','5'],avg,'-o')
    axs[1].plot(['BL','1','2','3','4','5'],avg2,'-o')
    axs[0].legend(['Symm','medium ASYMM with not so low axial','medium ASYMM with low axial','Highly Asymm'],loc='best')
    axs[1].legend(['Symm','medium ASYMM with not so low axial','medium ASYMM with low axial','Highly Asymm'],loc='best')
    axs[0].fill_between(['BL','1','2','3','4','5'],np.array(avg)-np.array(serr),np.array(avg)+np.array(serr),alpha=0.2)
    axs[1].fill_between(['BL','1','2','3','4','5'],np.array(avg2)-np.array(serr2),np.array(avg2)+np.array(serr2),alpha=0.2)
    axs[0].set_ylabel('Difference of AVG from BL')
    axs[0].set_xlabel('Years')
    axs[0].set_ylim(-2,11)
    axs[0].set_title('Axial_Sub_2')
    axs[1].set_title('Append_Sub')
    axs[1].set_ylabel('Difference of AVG from BL')
    axs[1].set_xlabel('Years')
    axs[1].set_ylim(-2,11)
    fig.tight_layout()
    fig.suptitle('Axial vs Appendicular ')
    fig.subplots_adjust()
        

In [ ]:
avg=[]
serr=[]
avg2=[]
serr2=[]
for k in years:
    patients=k['Axial_Sub_2']
    patients=patients[~np.isnan(patients)].to_list()
    avg.append(np.nanmean(patients))
    serr.append(sem(patients))
    
    patients2=k['Append_Sub']
    patients2=patients2[~np.isnan(patients2)].to_list()
    avg2.append(np.nanmean(patients2))
    serr2.append(sem(patients2))
    
a=[0]
avg=a+[i-avg[0] for i in avg[1:]] 
avg2=a+[i-avg2[0] for i in avg2[1:]] 
plt.figure(figsize=(10,5))
plt.plot(['BL','1','2','3','4','5'],avg,'-o')
plt.plot(['BL','1','2','3','4','5'],avg2,'-o')
plt.legend(['Axial_Sub_2','Appendicular_sub'],loc='best')
plt.title('De Novo Patients')

In [ ]:
Hc_BL_all=pd.read_csv("Desktop/Thesis/PPMI_Dataset/Evolution/HC/HC_BL_Features.csv")
Hc_BL_all.drop(columns="Unnamed: 0",axis=1,inplace=True)
Hc_V04=pd.read_csv("Desktop/Thesis/PPMI_Dataset/Evolution/HC/HC_V04_Features.csv")
Hc_V04.drop(columns="Unnamed: 0",axis=1,inplace=True)
Hc_V06=pd.read_csv("Desktop/Thesis/PPMI_Dataset/Evolution/HC/HC_V06_Features.csv")
Hc_V06.drop(columns="Unnamed: 0",axis=1,inplace=True)
Hc_V08=pd.read_csv("Desktop/Thesis/PPMI_Dataset/Evolution/HC/HC_V08_Features.csv")
Hc_V08.drop(columns="Unnamed: 0",axis=1,inplace=True)
Hc_V10=pd.read_csv("Desktop/Thesis/PPMI_Dataset/Evolution/HC/HC_V10_Features.csv")
Hc_V10.drop(columns="Unnamed: 0",axis=1,inplace=True)
Hc_V12=pd.read_csv("Desktop/Thesis/PPMI_Dataset/Evolution/HC/HC_V12_Features.csv")
Hc_V12.drop(columns="Unnamed: 0",axis=1,inplace=True)

In [ ]:
Hc_BL=scores(Hc_BL_all,'HC')
Hc_year1=scores(Hc_V04,'HC')
Hc_year2=scores(Hc_V06,'HC')
Hc_year3=scores(Hc_V08,'HC')
Hc_year4=scores(Hc_V10,'HC')
Hc_year5=scores(Hc_V12,'HC')


In [ ]:
hc_years=[Hc_BL,Hc_year1,Hc_year2,Hc_year3,Hc_year4,Hc_year5]
avg=[]
serr=[]
avg2=[]
serr2=[]
for k in hc_years:
    patients=k['Axial_Sub_2']
    patients=patients[~np.isnan(patients)].to_list()
    avg.append(np.nanmean(patients))
    serr.append(sem(patients))
    
    patients2=k['Append_Sub']
    patients2=patients2[~np.isnan(patients2)].to_list()
    avg2.append(np.nanmean(patients2))
    serr2.append(sem(patients2))
    
a=[0]
avg=a+[i-avg[0] for i in avg[1:]] 
avg2=a+[i-avg2[0] for i in avg2[1:]] 
plt.figure(figsize=(10,5))
plt.plot(['BL','1','2','3','4','5'],avg,'-o')
plt.plot(['BL','1','2','3','4','5'],avg2,'-o')
plt.legend(['Axial_Sub_2','Appendicular_sub'],loc='best')
plt.title('Healty Control')

In [ ]:
from PIL import Image
from sklearn.feature_selection import f_regression
from statsmodels.regression.linear_model import OLS
plt.rcParams.update({'font.size': 22})
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm

def lin_reg(start,yr,var,year,start_period,part):
    a=yr[np.isnan(yr[part])==False].reset_index()
    a=a.iloc[:,1:]
    temp=(a.loc[:,['PATNO',part]]).merge(start.loc[:,['PATNO',part]],on='PATNO',how='left',suffixes=('_'+year,'_BL'))
    temp[year+'-BL']=temp[part+'_'+year]-temp[part+'_BL']
    temp=temp.merge(start.loc[:,['PATNO',var]],on='PATNO',how='inner')
    temp=temp.dropna()
    y=temp[year+'-BL'].values
    x=temp[var].values
    f, ax = plt.subplots(1,1,figsize=(10,8))
    ax.plot(x+np.random.randn(len(x)),y+np.random.randn(len(y)),color='darkgray',marker='.',linestyle='None')
    ax.set_xlabel(var)
    ax.set_ylabel(year+'- '+start_period+' '+part)
    model=linear_model.LinearRegression()
    model.fit(x.reshape(-1,1),y)
    y_pred=(x.reshape(-1,1)*model.coef_+model.intercept_)
    ax.plot(x,y_pred,'k-')
    #plt.text(1,1,'R_Squared: '+str(np.round(metrics.r2_score(y,y_pred),4)))
    anchored_text = AnchoredText('R\N{SUPERSCRIPT TWO}= '+str(np.round(metrics.r2_score(y,y_pred),4)), loc=2)
    ax.add_artist(anchored_text)
    ax.set_ylim(-30,70)
    #ax.grid()
    if part=='Total_UPDRS3':
        X2 = sm.add_constant(x)
        est = OLS(y, X2)
        est2 = est.fit()
        print(est2.summary())
        plt.xticks(np.arange(0,80,20))
        ax.set_ylabel('Motor progression of PD at year 1')
        ax.set_xlabel('DatSCAN asymmetry index')
        #f.savefig('/Users/marco/Desktop/'+part+year+'.eps',format='eps')
        
        

# Prediction: Linear Rgression after 3 years
## from the results obtained we can say that the progression of the motor symptoms of the disease doesn't depend linearly from the symmetric index i.e doesn't depend linearly from the asymmetry, the score of the whole updrs 3 at BL and from the Axial Subscore.

In [ ]:
BL['Asymmetry']
x=pd.read_csv('../PPMI_Dataset/Subject_Characteristics/PPMI_Baseline_Data_02Jul2018.csv')
x=x[x['APPRDX']==1]
BL=BL.merge(x.loc[:,['PATNO','CAUDATE_L','CAUDATE_R','ai_caudate','ai_putamen','ai_striatum']],on='PATNO',how='left')
BL



In [ ]:
df=BL.loc[:,['PATNO','Total_UPDRS1','Total_UPDRS2','Total_UPDRS3','Asymmetry','ai_striatum']]
df1=year1.loc[:,['PATNO','Total_UPDRS1','Total_UPDRS2','Total_UPDRS3']]
df3=year3.loc[:,['PATNO','Total_UPDRS1','Total_UPDRS2','Total_UPDRS3']]
df5=year5.loc[:,['PATNO','Total_UPDRS1','Total_UPDRS2','Total_UPDRS3']]
df=df.merge(df1,on='PATNO',how='left',suffixes=("_BL", "_1yr"))
df=df.merge(df3,on='PATNO',how='left')
df=df.merge(df5,on='PATNO',how='left',suffixes=("_3yr", "_5yr"))
df

In [ ]:
ppp=df.iloc[:,[0,4,5,-9,-8,-7,-6,-5,-4,-3,-2,-1]]
pr=ppp.loc[:,['ai_striatum','3yr-BL_UPDRS3']].dropna()
x=pr[pr['ai_striatum']!='.']['ai_striatum']
y=pr['3yr-BL_UPDRS3']
print(x)
X2 = sm.add_constant(x)
est = OLS(y, X2)
est2 = est.fit()
print(est2.summary())


In [ ]:
plt.hist(BL['Asymmetry']/(BL['Left_Motor_Score']+BL['Right_Motor_Score']))
plt.xlabel('Normalized asymmetry')
plt.ylabel('Patients number')

In [ ]:
x=pd.read_csv('Desktop/DATScan_Analysis.csv')
x[x['PATNO']==4124]

In [ ]:
BL['ai_caudate']=BL['ai_caudate'].replace('.',np.nan)
BL['ai_caudate']=BL['ai_caudate'].astype('float64')
BL['ai_striatum']=BL['ai_striatum'].replace('.',np.nan)
BL['ai_striatum']=BL['ai_striatum'].astype('float64')
BL['ai_putamen']=BL['ai_putamen'].replace('.',np.nan)
BL['ai_putamen']=BL['ai_putamen'].astype('float64')
for i in BL.iloc[:,61:82].columns:
    lin_reg(BL,year5,'ai_striatum','5yr','BL',i)

    


In [ ]:
from scipy.stats import pearsonr
plt.figure()
plt.scatter(BL['ai_striatum'],BL['Asymmetry'])
a=BL.loc[:,['ai_striatum','Asymmetry']].dropna()
print(pearsonr(a['ai_striatum'],a['Asymmetry']))

plt.figure()
plt.scatter(BL['ai_caudate'],BL['Asymmetry'])
a=BL.loc[:,['ai_caudate','Asymmetry']].dropna()
print(pearsonr(a['ai_caudate'],a['Asymmetry']))

plt.figure()
plt.scatter(BL['ai_putamen'],BL['Asymmetry'])
a=BL.loc[:,['ai_putamen','Asymmetry']].dropna()
print(pearsonr(a['ai_putamen'],a['Asymmetry']))

In [ ]:
lin_reg(BL,year5,'Abs_Diff_part3','5yr','BL','Total_UPDRS3')

In [ ]:
lin_reg(BL,year5,'Total_UPDRS3','3yr','BL','MCATOT')

In [ ]:
lin_reg(BL,year5,'Axial_Sub_2','3yr','2yr','MCATOT')


# Prediction: Linear Rgression after 5 years
## from the results obtained we can say that the progression of the motor symptoms of the disease doesn't depend linearly from the symmetric index i.e doesn't depend linearly from the asymmetry, the score of the whole updrs 3 at BL and from the Axial Subscore.

In [ ]:
lin_reg(BL,year5,'Abs_Diff_part3','5yr','MCATOT')

In [ ]:
lin_reg(BL,year5,'Total_UPDRS3','5yr','Total_UPDRS3')

In [ ]:
lin_reg(BL,year5,'Axial_Sub_2','5yr','MCATOT')

## I tried also iterative methods like SGD or GD to fit the parameter, but clearly is not linear. 
# Combination of two or more variables

In [ ]:
def Multi_var_lin_reg(start,yr,var,year,part):
    
    
    
    a=yr[np.isnan(yr[part])==False].reset_index()
    a=a.iloc[:,1:]
    temp=(a.loc[:,['PATNO',part]]).merge(start.loc[:,['PATNO',part]],on='PATNO',how='left',suffixes=('_'+year,'_BL'))
    temp[year+'-BL']=temp[part+'_'+year]-temp[part+'_BL']
    temp=temp.merge(start.loc[:,['PATNO']+var],on='PATNO',how='left')
    temp=temp.dropna()
    y=temp[year+'-BL']
    x=temp[var]
    
    
    X_train, X_test, y_train, y_test=model_selection.train_test_split(x,y,test_size=0.2,shuffle=True)
    model=linear_model.LinearRegression()
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print('MSE:',metrics.mean_squared_error(y_test,y_pred))

## 3 Years Linear Regression multivariable

In [ ]:
Multi_var_lin_reg(BL,year5,['Abs_Diff_part3','Total_UPDRS3'],'3yr','MCATOT')
Multi_var_lin_reg(BL,year5,['Abs_Diff_part3','Axial_Sub_2'],'3yr','MCATOT')
Multi_var_lin_reg(BL,year5,['Total_UPDRS3','Axial_Sub_2'],'3yr','MCATOT')
Multi_var_lin_reg(BL,year5,['Total_UPDRS3','Axial_Sub_2','Abs_Diff_part3'],'3yr','MCATOT')

## 5 Years Linear Regression multivariable

In [ ]:
Multi_var_lin_reg(BL,year5,['Abs_Diff_part3','Total_UPDRS3'],'5yr','Total_UPDRS3')
Multi_var_lin_reg(BL,year5,['Abs_Diff_part3','Axial_Sub_2'],'5yr','Total_UPDRS3')
Multi_var_lin_reg(BL,year5,['Total_UPDRS3','Axial_Sub_2'],'5yr','Total_UPDRS3')
Multi_var_lin_reg(BL,year5,['Total_UPDRS3','Axial_Sub_2','Abs_Diff_part3'],'5yr','Total_UPDRS3')

In [ ]:
Multi_var_lin_reg(BL,year3,['Abs_Diff_part3','Total_UPDRS3'],'3yr','MCATOT')
Multi_var_lin_reg(BL,year3,['Abs_Diff_part3','Axial_Sub_2'],'3yr','MCATOT')
Multi_var_lin_reg(BL,year3,['Total_UPDRS3','Axial_Sub_2'],'3yr','MCATOT')
Multi_var_lin_reg(BL,year3,['Total_UPDRS3','Axial_Sub_2','Abs_Diff_part3'],'3yr','MCATOT')

# PIGD and TD

In [ ]:
year=year3
for var in BL.iloc[:,61:82].columns:

    pigd=BL[BL['PIGD_TD']=='PIGD']
    td=BL[BL['PIGD_TD']=='TD']
    ind=BL[BL['PIGD_TD']=='IND']

    pigd['Asymmetry']
    pigd_yr=year[year['PATNO'].isin(pigd['PATNO'])].loc[:,['PATNO',var]].dropna()
    td_yr=year[year['PATNO'].isin(td['PATNO'])].loc[:,['PATNO',var]].dropna()
    ind_yr=year[year['PATNO'].isin(ind['PATNO'])].loc[:,['PATNO',var]].dropna()




    pigd_BL=BL[BL['PATNO'].isin(pigd_yr['PATNO'])][var]
    td_BL=BL[BL['PATNO'].isin(td_yr['PATNO'])][var]
    ind_BL=BL[BL['PATNO'].isin(ind_yr['PATNO'])][var]


    td_prog=(td_yr[var].to_numpy()-td_BL.to_numpy())
    pigd_prog=(pigd_yr[var].to_numpy()-pigd_BL.to_numpy())
    ind_prog=(ind_yr[var].to_numpy()-ind_BL.to_numpy())

    print(var+'\n')
    print('PIGD vs TD')
    print(stats.kruskal(pigd_prog,td_prog))
    print('TD vs IND')
    print(stats.kruskal(td_prog,ind_prog))
    print('IND vs PIGD')
    print(stats.kruskal(ind_prog,pigd_prog))
    print('\n')


In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Motor_Assessments/MDS_UPDRS_Part_I.csv')
x=x[x['PATNO'].isin(BL['PATNO'])]
x=x[x['EVENT_ID']=='V05']
x
#year2['Total_UPDRS1']

In [ ]:

FILE = "Desktop/New_Asym_vs_sym.html"

with open(FILE, 'r') as html_file:
    content = html_file.read()

# Get rid off prompts and source code
content = content.replace("div.input_area {","div.input_area {\n\tdisplay: none;")    
content = content.replace(".prompt {",".prompt {\n\tdisplay: none;")

f = open(FILE, 'w')
f.write(content)
f.close()